# Tarea 2 – ISIS 4221, Notebook II

## Paso 0: Preparacion

In [1]:
from load_ngram_model import load_ngram_model
import numpy as np
import pandas as pd

In [2]:
# Load the models for 20N dataset
unigram_counts_20n, unigram_20n = load_ngram_model('20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_unigrams')
bigram_counts_20n, bigram_20n = load_ngram_model('20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_bigrams')
trigram_counts_20n, trigram_20n = load_ngram_model('20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_trigrams')

# Load the models for BAC dataset
unigram_counts_bac, unigram_bac = load_ngram_model('BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_unigrams')
bigram_counts_bac, bigram_bac = load_ngram_model('BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_bigrams')
trigram_counts_bac, trigram_bac = load_ngram_model('BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_trigrams')

Modelo 20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_unigrams cargado exitosamente.
Modelo 20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_bigrams cargado exitosamente.
Modelo 20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_trigrams cargado exitosamente.
Modelo BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_unigrams cargado exitosamente.
Modelo BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_bigrams cargado exitosamente.
Modelo BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_trigrams cargado exitosamente.


In [4]:
# Load the 20N testing dataset
df_news_test = pd.read_parquet('data/train_test/20N_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_testing.parquet')
df_news_test.head()

,text,source,category,length
0,<s> atheist resources. </s>,./raw_data/20news-18828\alt.atheism\49960,alt.atheism,4
1,<s> NUM NUM 8900.evolution designs.evolution d...,./raw_data/20news-18828\alt.atheism\49960,alt.atheism,11
2,<s> ball and g.w. </s>,./raw_data/20news-18828\alt.atheism\49960,alt.atheism,5
3,<s> american atheist press. </s>,./raw_data/20news-18828\alt.atheism\49960,alt.atheism,5
4,<s> isbn NUM 910309 NUM NUM 2nd edition 1986. ...,./raw_data/20news-18828\alt.atheism\49960,alt.atheism,18


In [5]:
# Load the bac testing dataset
df_bac_test = pd.read_parquet('data/train_test/BAC_CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba_testing.parquet')
df_bac_test.head()

,text,source,length
0,<s> this has been a lot of rain though. </s>,./raw_data/blogs\1000331.female.37.indUnk.Leo.xml,10
1,<s> but it is supposed to be NUM degrees by fr...,./raw_data/blogs\1000331.female.37.indUnk.Leo.xml,20
2,<s> shell say mom. </s>,./raw_data/blogs\1000331.female.37.indUnk.Leo.xml,5
3,<s> this is why i go to work. </s>,./raw_data/blogs\1000331.female.37.indUnk.Leo.xml,9
4,<s> we used to go to the jays games occassiona...,./raw_data/blogs\1000331.female.37.indUnk.Leo.xml,27


## Paso 1: Definición de las Funciones Necesarias

In [11]:
def estimate_probability(token_text: list[str], n_gram: int, 
                         final_unigram: dict, ngram_counts: dict) -> float:
    """
    Estima la probabilidad de un n-grama dado usando un modelo de n-gramas.

    Args:
        token_text (list[str]): La secuencia de tokens para la cual se desea estimar la probabilidad.
        n_gram (int): El tamaño del n-grama.
        final_unigram (dict): El diccionario que contiene las frecuencias de los (n-1)-gramas.
        ngram_counts (dict): El diccionario que contiene las frecuencias de los n-gramas.

    Raises:
        ValueError: Si el número de tokens en `token_text` no coincide con `n_gram`.

    Returns:
        float: La probabilidad estimada del n-grama dado.

    Notas:
        - La función utiliza suavizado de Laplace para evitar probabilidades de cero.
        - Si el prefijo (n-1)-grama no se encuentra en `final_unigram`, se reemplaza con (`<UNK>`,)* (n_gram-1).
    """    
    
    if len(token_text) != n_gram:
        raise ValueError(f'El texto de entrada debe tener {n_gram} tokens')
    
    # Reemplaza el prefijo con `<UNK>` si no se encuentra en final_unigram
    if not final_unigram.get(tuple(token_text[:n_gram-1])):
        token_text = list(token_text)
        token_text[:n_gram] = ('<UNK>',) * (n_gram-1)
        token_text = tuple(token_text)

    # Suavizado de Laplace
    # Probabilidad del (n-1)-grama más tamaño del vocabulario
    p_wi = final_unigram.get(tuple(token_text[:n_gram-1]), 0) + len(final_unigram)
    
    # Probabilidad del n-grama completo dado su (n-1)-grama
    p_w_wi = (ngram_counts.get(tuple(token_text), 1)) / p_wi
        
    return p_w_wi

def compute_perplexity(test_set, ngram_counts, final_unigram, n_gram):
    """
    Calcula la perplejidad de un conjunto de prueba usando el modelo de n-gramas.

    La perplejidad es una métrica que evalúa qué tan bien el modelo predice el texto de prueba. 
    Valores más bajos de perplejidad indican mejores predicciones.

    Args:
        test_set (list): El conjunto de oraciones de prueba.
        ngram_counts (dict): El modelo de n-gramas con las frecuencias de los n-gramas.
        final_unigram (dict): El modelo de (n-1)-gramas con las frecuencias.
        n_gram (int): El tamaño del n-grama (1 para unigramas, 2 para bigramas, etc.).

    Returns:
        float: El valor de la perplejidad calculada para el conjunto de prueba.
    """
    perplexity = 0  # Inicializa la suma acumulativa de la perplejidad.
    total_words = 0  # Contador para el número total de palabras procesadas.
    
    for sentence in test_set:
        tokens = sentence.split()  # Divide la oración en tokens.
        
        # Recorre los tokens desde el índice n_gram - 1 hasta el final.
        for i in range(n_gram - 1, len(tokens)):
            # Estima la probabilidad del n-grama actual.
            prob = estimate_probability(tokens[i-n_gram+1:i+1], n_gram, final_unigram, ngram_counts)
            # Calcula la logaritmo negativo de la probabilidad y la suma a la perplejidad.
            perplexity -= np.log(prob)
        
        # Incrementa el número total de palabras procesadas.
        total_words += len(tokens) - (n_gram - 1)
    
    # Devuelve la perplejidad exponencial media.
    return np.exp(perplexity / total_words)

def generate_sentence(start_word, ngram_counts, n_gram, max_length=15):
    """
    Genera una oración automáticamente a partir de un modelo de n-gramas.

    Esta función toma una palabra inicial y construye una oración basada en los n-gramas más frecuentes
    del modelo, generando hasta `max_length` palabras.

    Args:
        start_word (str): La primera palabra de la oración.
        ngram_counts (dict): El modelo de n-gramas con las frecuencias de los n-gramas.
        final_unigram (dict): El modelo de (n-1)-gramas con las frecuencias.
        n_gram (int): El tamaño del n-grama utilizado (1 para unigramas, 2 para bigramas, etc.).
        max_length (int): La longitud máxima de la oración generada.

    Returns:
        str: La oración generada automáticamente.
    """
    sentence = [start_word]  # Inicializa la oración con la palabra inicial.
    
    for _ in range(max_length):
        # Encuentra los candidatos a la siguiente palabra basados en el contexto actual.
        next_token_candidates = [(ngram[-1], count) for ngram, count in ngram_counts.items()
                                 if ngram[:-1] == tuple(sentence[-(n_gram-1):])]
        
        if not next_token_candidates:
            break  # Sale del bucle si no hay candidatos válidos.
        
        # Selecciona la siguiente palabra con la mayor frecuencia.
        next_token = max(next_token_candidates, key=lambda x: x[1])[0]
        sentence.append(next_token)  # Añade la palabra seleccionada a la oración.
        
        if next_token == '</s>':
            break  # Termina la oración si se encuentra el token de finalización.
    
    return ' '.join(sentence)  # Devuelve la oración generada como una cadena de texto.

## Paso 2: Cálculo de la Perplejidad de los Modelos de N-gramas de 20N y BAC

In [8]:
# Calcula la perplejidad de los modelos de N-gramas de 20N
perplexity_unigram_20n = compute_perplexity(df_news_test['text'], unigram_counts_20n, unigram_20n, 1)
perplexity_bigram_20n = compute_perplexity(df_news_test['text'], bigram_counts_20n, bigram_20n, 2)
perplexity_trigram_20n = compute_perplexity(df_news_test['text'], trigram_counts_20n, trigram_20n, 3)

print(f"Perplejidad (Unigrama 20N): {perplexity_unigram_20n}")
print(f"Perplejidad (Bigrama 20N): {perplexity_bigram_20n}")
print(f"Perplejidad (Trigrama 20N): {perplexity_trigram_20n}")

Perplejidad (Unigrama 20N): 1.0
Perplejidad (Bigrama 20N): 4683.496965341476
Perplejidad (Trigrama 20N): 168769.4673538683


In [9]:
# Calcula la perplejidad de los modelos de N-gramas de BAC
perplexity_unigram_bac = compute_perplexity(df_bac_test['text'], unigram_counts_bac, unigram_bac, 1)
perplexity_bigram_bac = compute_perplexity(df_bac_test['text'], bigram_counts_bac, bigram_bac, 2)
perplexity_trigram_bac = compute_perplexity(df_bac_test['text'], trigram_counts_bac, trigram_bac, 3)

print(f"Perplejidad (Unigrama BAC): {perplexity_unigram_bac}")
print(f"Perplejidad (Bigrama BAC): {perplexity_bigram_bac}")
print(f"Perplejidad (Trigrama BAC): {perplexity_trigram_bac}")

Perplejidad (Unigrama BAC): 1.0
Perplejidad (Bigrama BAC): 1596.4153151273158
Perplejidad (Trigrama BAC): 267404.24922253174


## Paso 3: Generación de Sentencias con los Mejores Modelos de N-gramas de 20N y BAC

In [27]:
# Ejemplo de generación de oraciones con `bigram_counts_20n`
generated_sentence_20n = generate_sentence('<s>', bigram_counts_20n, 2, 10)
print(f"Oración generada (20N): {generated_sentence_20n}")

Oración generada (20N): <s> i have a ('<UNK>',) ('<UNK>',) ('<UNK>',) ('<UNK>',) ('<UNK>',) ('<UNK>',) ('<UNK>',)


In [28]:
# Ejemplo de generación de oraciones con `bigram_counts_bac`
generated_sentence_bac = generate_sentence('book', bigram_counts_bac, 2, 10)
print(f"Oración generada (BAC): {generated_sentence_bac}")

Oración generada (BAC): book and i was a little bit of the same time


In [29]:
# Ejemplo de generación de oraciones con `bigram_counts_bac`
generated_sentence_bac = generate_sentence('tonight', bigram_counts_bac, 2, 10)
print(f"Oración generada (BAC): {generated_sentence_bac}")

Oración generada (BAC): tonight i was a little bit of the same time to


In [30]:
# Ejemplo de generación de oraciones con `bigram_counts_bac`
generated_sentence_bac = generate_sentence('hilarious', bigram_counts_bac, 2, 10)
print(f"Oración generada (BAC): {generated_sentence_bac}")

Oración generada (BAC): hilarious and i was a little bit of the same time


In [37]:
# Ejemplo de generación de oraciones con `bigram_counts_bac`
generated_sentence_bac = generate_sentence('he', bigram_counts_bac, 2, 10)
print(f"Oración generada (BAC): {generated_sentence_bac}")

Oración generada (BAC): he was a little bit of the same time to the


In [38]:
# Ejemplo de generación de oraciones con `bigram_counts_bac`
generated_sentence_bac = generate_sentence('</s>', bigram_counts_bac, 2, 10)
print(f"Oración generada (BAC): {generated_sentence_bac}")

Oración generada (BAC): </s>


In [39]:
# Ejemplo de generación de oraciones con `bigram_counts_bac`
generated_sentence_bac = generate_sentence('<s>', bigram_counts_bac, 2, 10)
print(f"Oración generada (BAC): {generated_sentence_bac}")

Oración generada (BAC): <s> i was a little bit of the same time to
